In [26]:
import xgboost as xgb
import numpy as np
import pandas as pd
import time
import pickle
import os
from math import log
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, ParameterGrid,StratifiedKFold, cross_val_score

from sklearn import metrics 
from sklearn.metrics import confusion_matrix, precision_recall_curve, average_precision_score, brier_score_loss, f1_score, log_loss,auc,roc_curve,auc

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier

from sklearn.decomposition import PCA
#from sklearn.utils.fixes import signature

from datetime import datetime

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
os.chdir(r"C:\Users\aman.khatri91425\Desktop\HomeCredit\AAM\False Rejects")

In [3]:
df=pd.read_csv(os.getcwd()+"\Data Files\OP180NVHMaint.csv")

In [4]:
df.head(5)

,SITENAME,LINENAME,STATIONNAME,PARTNUMBER,STATUS,TOPLEVELSERIALNUMBER,PROD_TIMESTAMP,COLLID,CHARDESC,CHARVALUE
0,GMCFULL,T1XX Front Axle FA HiV,OP180B,40217205,A,01A200150794,2020-01-17 05:57:44.613,-1281960286,NVH Torque Sweep Test Cycle Count,1.0
1,GMCFULL,T1XX Front Axle FA HiV,OP180B,40217205,A,01A200120004,2020-01-16 00:48:22.773,-1282718562,NVH Torque Sweep Test Cycle Count,1.0
2,GMCFULL,T1XX Front Axle FA HiV,OP180B,40217205,A,01A200130594,2020-01-16 01:34:25.477,-1282699737,NVH Torque Sweep Test Cycle Count,1.0
3,GMCFULL,T1XX Front Axle FA HiV,OP180A,40217205,A,01A200150767,2020-01-17 00:51:39.730,-1282102412,NVH Torque Sweep Test Cycle Count,1.0
4,GMCFULL,T1XX Front Axle FA HiV,OP180A,40217205,A,01A200140383,2020-01-16 03:31:51.810,-1282651218,NVH Torque Sweep Test Cycle Count,1.0


In [23]:
df["PROD_TIMESTAMP"] = pd.to_datetime(df["PROD_TIMESTAMP"],format = '%Y-%m-%d %H:%M:%S.%f')

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 437607 entries, 0 to 437606
Data columns (total 10 columns):
SITENAME                437607 non-null object
LINENAME                437607 non-null object
STATIONNAME             437607 non-null object
PARTNUMBER              437607 non-null int64
STATUS                  437607 non-null object
TOPLEVELSERIALNUMBER    437607 non-null object
PROD_TIMESTAMP          437607 non-null datetime64[ns]
COLLID                  437607 non-null int64
CHARDESC                437607 non-null object
CHARVALUE               437607 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 33.4+ MB


In [25]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
PARTNUMBER,437607.0,4.021721e+07,4.484519e-01,4.021720e+07,4.021720e+07,4.021720e+07,4.021720e+07,4.021721e+07
COLLID,437607.0,-1.248481e+09,1.176665e+07,-1.283455e+09,-1.253534e+09,-1.246316e+09,-1.239562e+09,-1.233079e+09
CHARVALUE,437607.0,5.395492e+02,5.678288e+02,0.000000e+00,1.000000e+00,2.500270e+02,1.095470e+03,1.500000e+03


In [7]:
# import pandas_profiling
from pandas_profiling import ProfileReport
prof = ProfileReport(df)
prof.to_file(outputfile='output.html')
# df.profile_report()

In [8]:
df[["CHARDESC","CHARVALUE"]].groupby("CHARDESC").agg(["max","min"])

CHARVALUE          
                                        max       min
CHARDESC                                             
LH Slide To Full Depth Posn        1096.970  1081.560
NVH Torque Sweep Test Cycle Count     2.000     0.000
Pinion Slide To Full Depth Posn     252.934   242.427
RH Slide To Full Depth Posn        1500.000  1106.580

In [9]:
station_count_df=df[["STATIONNAME","CHARVALUE"]].groupby("STATIONNAME").agg(["count","max","min","mean","median"]).reset_index()
station_count_df.columns = ["STATIONNAME","COUNT","MAX_CHARVALUE","MIN_CHAR_VALUE","MEAN_CHAR_VALUE","MEDIAN_CHAR_VALUE"]

station_count_df[["STATIONNAME","COUNT"]].plot(kind = 'bar', x='STATIONNAME')

station_count_df

,STATIONNAME,COUNT,MAX_CHARVALUE,MIN_CHAR_VALUE,MEAN_CHAR_VALUE,MEDIAN_CHAR_VALUE
0,OP180A,232565,1500.00,0.0,575.796138,245.116
1,OP180B,205042,1117.33,0.0,498.436747,251.868


In [10]:
station_count_df

,STATIONNAME,COUNT,MAX_CHARVALUE,MIN_CHAR_VALUE,MEAN_CHAR_VALUE,MEDIAN_CHAR_VALUE
0,OP180A,232565,1500.00,0.0,575.796138,245.116
1,OP180B,205042,1117.33,0.0,498.436747,251.868


In [11]:
status_summ_df=df[["STATUS","CHARVALUE"]].groupby("STATUS").agg(["count","max","min","mean","median"]).reset_index()
status_summ_df.columns = ["STATUS","COUNT","MAX_CHARVALUE","MIN_CHAR_VALUE","MEAN_CHAR_VALUE","MEDIAN_CHAR_VALUE"]
status_summ_df

,STATUS,COUNT,MAX_CHARVALUE,MIN_CHAR_VALUE,MEAN_CHAR_VALUE,MEDIAN_CHAR_VALUE
0,A,437024,1500.00,1.0,539.538550,250.0075
1,X,583,1498.37,0.0,547.514556,251.5970


In [12]:
fig = plt.figure(figsize = (10,3))
ax1= fig.add_subplot(1,2,1)
df[df["STATUS"]=="A"].CHARVALUE.hist(density=True, bins=50)
plt.title("CHARVALUE Dist A")
plt.xticks(rotation = 90,)

ax2= fig.add_subplot(1,2,2)
df[df["STATUS"]=="X"].CHARVALUE.hist(density=True, bins = 50)
plt.title("CHARVALUE Dist X")
plt.xticks(rotation = 90,)


plt.show()

C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


In [13]:
df[df["STATUS"]=="A"].CHARVALUE.hist(density=True)

##### Both accepted and rejected parts have similar trends in CHar_value

In [14]:
df[df["CHARVALUE"]==0].STATUS.value_counts()

X    9
Name: STATUS, dtype: int64

##### 0 Charvalue corresponds only to rejected parts

In [15]:
df_tmp = df["COLLID"].value_counts()
df_tmp.hist()

In [18]:
df["PROD_TIMESTAMP"].value_counts().value_counts()

4    88820
1    82310
3        3
8        1
Name: PROD_TIMESTAMP, dtype: int64